# WGAN-GP in Pytorch

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm
import torch.autograd as autograd
from scipy import stats
from torch.utils.tensorboard import SummaryWriter
from typing import Tuple, List, Type, Dict, Any
from os.path import join, isfile, isdir
from queue import Empty, Queue
from threading import Thread

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [3]:
##################################
### Здесь есть ошибка! Какая? ####
##################################

class Generator(nn.Module):

    def __init__(self, n_inputs=128):
        super(Generator, self).__init__()

        self.fc1 = nn.Linear(n_inputs, 512)
        self.bn_fc1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 1024)


    def forward(self, x_noise):
        x = x_noise  # N x n_inputs
        
        x = F.relu(self.bn_fc1(self.fc1(x)))  # N x 512
        x = F.relu(self.fc2(x))               # N x 1024
        
        x = torch.sigmoid(x)

        return x

In [4]:
##############################################################
### Здесь есть ошибка (при условии, что это WGAN)! Какая? ####
##############################################################

class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()

        self.fc1 = nn.Linear(1024, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
##################################
### Здесь есть ошибка! Какая? ####
##################################

gen = Generator(n_inputs=latent_dim)
gen = gen.to(DEVICE)

dsc = Discriminator()
dsc = dsc.to(DEVICE)

optimizer_gen = torch.optim.Adam(gen.parameters(), lr=1e-4)
optimizer_dsc = torch.optim.SGD(dsc.parameters(), lr=1e-4, momentum=0.9)

In [ ]:
##################################
###        Train loop         ####
##################################

for epoch in range(max_epochs):
    print(f'Starting epoch {epoch + 1} of {max_epochs}')

    gen.train()
    dsc.train()

    with tqdm(total=STEPS_PER_EPOCH) as pbar:
        for idx in range(STEPS_PER_EPOCH):
            # Training discriminator
            optimizer_dsc.zero_grad()

            real_batch = next(train_ds).to(DEVICE)
            noise = Variable(torch.tensor(np.random.normal(0, 1, (len(real_batch), latent_dim)), dtype=torch.float, device=DEVICE))
            fake_batch = gen(noise)
            
            fake_batch = torch.mul(fake_batch, mask_batch)

            real_scores = dsc(real_batch)
            fake_scores = dsc(fake_batch)
            gradient_penalty = compute_gradient_penalty(dsc, real_batch, fake_batch)

            loss_dsc = -torch.mean(real_scores) + torch.mean(fake_scores) + lambda_gp * gradient_penalty

            loss_dsc.backward()
            optimizer_dsc.step()

            # Training generator
            optimizer_gen.zero_grad()
            fake_batch = gen(noise)
            fake_batch = torch.mul(fake_batch, mask_batch)

            fake_scores = dsc(fake_batch)
            loss_gen = -torch.mean(fake_scores)

            loss_gen.backward()
            optimizer_gen.step()

            pbar.update(1)
            pbar.set_postfix({'step': idx + 1, 'loss_GEN': loss_gen.item(), 'loss_DSC': loss_dsc.item()})

    # estimating WGAN-GP loss after an epoch
    gen.eval()
    dsc.eval()
    val_loss_epoch = []

    with tqdm(total=VAL_STEPS) as pbar:
        for idx in range(VAL_STEPS):
            real_batch = next(train_ds).to(DEVICE)
            noise = Variable(torch.tensor(np.random.normal(0, 1, (len(real_batch), latent_dim)), dtype=torch.float, device=DEVICE))
            fake_batch = gen(noise)
            fake_batch = torch.mul(fake_batch, mask_batch)
            val_loss_epoch.append((torch.mean(dsc(real_batch) - dsc(fake_batch))).detach().cpu())
            pbar.update(1)

    val_loss_epoch = np.sum(val_loss_epoch)/(VAL_STEPS*VAL_BATCH_SIZE)

    lr_scheduler_gen.step()
    lr_scheduler_dsc.step()